# CLASSIFICAZIONE LOGHI FAKE E REAL
Utilizziamo un modello pre-addestrato per per distinguere loghi autentici da loghi falsi.
## 1. Importazione delle Librerie

Le seguenti librerie vengono importate per i seguenti scopi:
- **TensorFlow**: Fondamentale per il deep learning e la creazione di modelli di rete neurale.
- **NumPy**: Fornisce supporto per lavorare con array e operazioni matematiche avanzate.
- **Pillow (PIL)**: Per caricare e manipolare immagini.
- **Pandas**: Per la manipolazione dei dati e il caricamento di file CSV.
- **Scikit-learn**: Per generare i report di classificazione.
- **OS**: Per la gestione di file e directory nel sistema operativo.

In [1]:
import tensorflow as tf
import numpy as np 
from PIL import Image
import pandas as pd
from sklearn.metrics import classification_report
import os

## 2. Caricamento del Modello Pre-addestrato
In questa fase:
- Carichiamo il modello pre-addestrato utilizzando TensorFlow.
- Definiamo le classi di output (Fake e Genuine).

In [2]:
# Percorso del modello
model_path = '/Users/ilaria/ProgettoDTM/EfficientNet'

# Carica il modello
model = tf.saved_model.load(model_path)

# Definizione delle classi
classes = ["Fake", "Genuine"]

print("Modello caricato con successo!")

Modello caricato con successo!


## 3. Preparazione del Dataset di Training

### 3.1 Estrazione dei Percorsi delle Immagini
- Navigazione nelle directory per raccogliere i percorsi completi dei file.

### 3.2 Creazione del DataFrame
- Creazione di un DataFrame contenente i percorsi e le etichette (Fake o Genuine).

In [3]:
# Percorso locale ai tuoi file
base_path = '/Users/ilaria/ProgettoDTM/archive/train'  # Modifica questo percorso se necessario

# Crea una lista vuota chiamata 'Id' per memorizzare i percorsi dei file
Id = []

# Scorri i file nella directory di training
for dirname, _, filenames in os.walk(base_path):
    for filename in filenames:
        Id.append(os.path.join(dirname, filename)) # Per ogni file trovato, aggiunge il percorso completo del file alla lista 'Id'

# Visualizza i primi 5 percorsi
print(Id[:5])

# Crea un DataFrame vuoto 'train' utilizzando la libreria pandas
train=pd.DataFrame()

# Aggiunge una nuova colonna 'filename' coi percorsi dei file dalla lista 'Id', per associare ogni immagine al suo percorso nel DataFrame
train=train.assign(filename=Id)

# Visualizza le prime 5 righe per un'anteprima dei dati e verificare che 'filename' sia stata correttamente aggiunta
train.head()

# Crea una nuova colonna 'label' e inizializzala con i valori della colonna 'filename'
train['label'] = train['filename']

# Modifica i valori della colonna 'label' rimuovendo il prefisso dai percorsi per semplificare i nomi dei file
train['label'] = train['label'].str.replace('/Users/ilaria/ProgettoDTM/archive/train/', '', regex=False)

# Mostra le prime righe del DataFrame
train.head()

# Estrae la prima parte del percorso per ottenere la categoria del logo (Fake o Genuine)
train['label'] = train['label'].str.split('/').str[0]
train.head()

['/Users/ilaria/Progetto DTM/archive/train/Genuine/000004_9bf73fcef2974cb6a59c0e8bffb82064.jpg', '/Users/ilaria/Progetto DTM/archive/train/Genuine/000003_9b0a7ffb9cec41b2acd283ed706eb14c.jpg', '/Users/ilaria/Progetto DTM/archive/train/Genuine/000003_2a54cc642d684acc8f087051e6d47ef1.jpg', '/Users/ilaria/Progetto DTM/archive/train/Genuine/000004_8c95c1461b3f4f73b6d749e2326fbd92.jpg', '/Users/ilaria/Progetto DTM/archive/train/Genuine/000005_684ddfeb89c249a0b5739ed2c738e58c.jpg']


,filename,label
0,/Users/ilaria/Progetto DTM/archive/train/Genui...,Genuine
1,/Users/ilaria/Progetto DTM/archive/train/Genui...,Genuine
2,/Users/ilaria/Progetto DTM/archive/train/Genui...,Genuine
3,/Users/ilaria/Progetto DTM/archive/train/Genui...,Genuine
4,/Users/ilaria/Progetto DTM/archive/train/Genui...,Genuine


## 4. Preparazione del Dataset di Test

### 4.1 Estrazione dei Percorsi delle Immagini
- Raccolta dei percorsi dei file di test.

### 4.2 Creazione del DataFrame
- Creazione di un DataFrame con i percorsi e le etichette (Fake o Genuine).

In [4]:
# Operazione simile a quella precedente che si differenzia da quella effettuata per i dati di addestraemto perchè fa riferiemnto ai dati di test
Id = []

# Percorso locale per i dati di test
test_path = '/Users/ilaria/ProgettoDTM/archive/test'

# Itera sui file nella directory di test
for dirname, _, filenames in os.walk(test_path):
    for filename in filenames:
        Id.append(os.path.join(dirname, filename))  # Aggiunge i percorsi completi alla lista

# Mostra i primi 5 percorsi
print(Id[:5])

test=pd.DataFrame()
test=test.assign(filename=Id)
test.head()

test['label']=test['filename']
test['label']=test['label'].str.replace('/Users/ilaria/ProgettoDTM/archive/test/', '', regex=False)
test.head()

test['label'] = test['label'].str.split('/').str[0]
test.head()

['/Users/ilaria/Progetto DTM/archive/test/Genuine/000004_917c1f2045fa403eb2f2946d44eee319.jpg', '/Users/ilaria/Progetto DTM/archive/test/Genuine/000005_98a6283cd9ef4ba2a2511bca3e0efa92.jpg', '/Users/ilaria/Progetto DTM/archive/test/Genuine/000003_ce6327493dfb41a2b60dfd3389d91efa.jpg', '/Users/ilaria/Progetto DTM/archive/test/Genuine/000001_3272b402545540d5a7c90eae7bd402a6.jpg', '/Users/ilaria/Progetto DTM/archive/test/Genuine/000003_c99519bc79b2480b807f1fb4f09e0968.jpg']


,filename,label
0,/Users/ilaria/Progetto DTM/archive/test/Genuin...,Genuine
1,/Users/ilaria/Progetto DTM/archive/test/Genuin...,Genuine
2,/Users/ilaria/Progetto DTM/archive/test/Genuin...,Genuine
3,/Users/ilaria/Progetto DTM/archive/test/Genuin...,Genuine
4,/Users/ilaria/Progetto DTM/archive/test/Genuin...,Genuine


## 5. Preprocessing e Predizioni

In questa fase:
1. Effettuiamo il preprocessing delle immagini.
2. Generiamo le predizioni utilizzando il modello pre-addestrato.

### 5.1 Predizioni sul Dataset di Test
- Preprocessing delle immagini di test.
- Calcolo delle predizioni.

In [5]:
# Inizializza una lista per memorizzare le predizioni
result = []

# Loop su ogni immagine del test set
for i in test['filename']:
    # Apre l'immagine e la converte in RGB
    img = Image.open(i).convert('RGB')
    
    # Ridimensiona l'immagine mantenendo le proporzioni
    img = img.resize((300, 300 * img.size[1] // img.size[0]), Image.Resampling.LANCZOS)
    
    # Converte l'immagine in array numpy senza normalizzazione
    inp_numpy = np.array(img)[None]
    
    # Converte l'array numpy in un tensore TensorFlow
    inp = tf.constant(inp_numpy, dtype='float32')
    
    # Passa l'immagine al modello per ottenere le predizioni
    class_scores = model(inp)[0].numpy()
    
    # Determina la classe con il punteggio massimo
    result.append(classes[class_scores.argmax()])


# Preprocessing e predizioni per il test
result = []

for i in test['filename']:
    img = Image.open(i).convert('RGB')  # Apre l'immagine e la converte in RGB
    img = img.resize((300, 300 * img.size[1] // img.size[0]), Image.Resampling.LANCZOS)  # Ridimensiona mantenendo le proporzioni
    inp_numpy = np.array(img)[None]  # Converte l'immagine in un array numpy e aggiunge una dimensione batch
    inp = tf.constant(inp_numpy, dtype='float32')  # Converte l'array numpy in un tensore TensorFlow
    class_scores = model(inp)[0].numpy()  # Passa l'immagine nel modello per ottenere le probabilità di ciascuna classe
    result.append(classes[class_scores.argmax()])  # Aggiunge la classe predetta (quella con il punteggio più alto) alla lista 'result'

# Visualizza le prime 5 predizioni ottenute dal modello
print(result[:5])

# Aggiunge una colonna 'prediction' al DataFrame 'test', con le predizioni
test=test.assign(prediction=result)
test.head()

['Fake', 'Fake', 'Fake', 'Fake', 'Fake']


,filename,label,prediction
0,/Users/ilaria/Progetto DTM/archive/test/Genuin...,Genuine,Fake
1,/Users/ilaria/Progetto DTM/archive/test/Genuin...,Genuine,Fake
2,/Users/ilaria/Progetto DTM/archive/test/Genuin...,Genuine,Fake
3,/Users/ilaria/Progetto DTM/archive/test/Genuin...,Genuine,Fake
4,/Users/ilaria/Progetto DTM/archive/test/Genuin...,Genuine,Fake


### 5.2 Predizioni sul Dataset di Training
- Preprocessing delle immagini di training.
- Calcolo delle predizioni.

In [6]:
# Stesso procedimento ma effettuato sulle immagini di addestramento (train)
result=[]
for i in train.filename:
    img = Image.open(i).convert('RGB')
    img = img.resize((300, 300 * img.size[1] // img.size[0]), Image.Resampling.LANCZOS)
    inp_numpy = np.array(img)[None]
    inp = tf.constant(inp_numpy, dtype='float32')
    class_scores = model(inp)[0].numpy()
    result.append(classes[class_scores.argmax()])
result[:5]

# Aggiunge una colonna 'prediction' al DataFrame 'train', con le predizioni
train=train.assign(prediction=result)
train.head()

,filename,label,prediction
0,/Users/ilaria/Progetto DTM/archive/train/Genui...,Genuine,Genuine
1,/Users/ilaria/Progetto DTM/archive/train/Genui...,Genuine,Genuine
2,/Users/ilaria/Progetto DTM/archive/train/Genui...,Genuine,Genuine
3,/Users/ilaria/Progetto DTM/archive/train/Genui...,Genuine,Genuine
4,/Users/ilaria/Progetto DTM/archive/train/Genui...,Genuine,Genuine


## 6. Valutazione del Modello

Generiamo i report di classificazione per valutare le performance del modello su:
1. Dataset di Training
2. Dataset di Test

In [7]:
# Stampa il report confrontando le etichette vere ('label') con le predizioni del modello ('prediction')
print(classification_report(train['label'],train['prediction']))

# Stampa il report di classificazione confrontando le etichette vere e le predizioni sui dati di test per valutare le prestazioni del modello
print(classification_report(test['label'],test['prediction']))

              precision    recall  f1-score   support

        Fake       0.83      1.00      0.90       439
     Genuine       1.00      0.58      0.73       220

    accuracy                           0.86       659
   macro avg       0.91      0.79      0.82       659
weighted avg       0.88      0.86      0.85       659

              precision    recall  f1-score   support

        Fake       0.73      0.96      0.83       110
     Genuine       0.79      0.27      0.41        55

    accuracy                           0.73       165
   macro avg       0.76      0.62      0.62       165
weighted avg       0.75      0.73      0.69       165

